In [ ]:
import psycopg2

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=postgres password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

In [ ]:
#### Let's imagine we have a table called Music Store. 

#Table Name: music_store
#column 0: Transaction Id
#column 1: Customer Name
#column 2: Cashier Name
#column 3: Year 
#column 4: Albums Purchased`


## Now to translate this information into a CREATE Table Statement and insert the data

#Tr Id     Customer Name    Cashier Name       Year    Albuns Purchased
#1            Amanda           Sam             2000  ['Rubber Soul', 'Let it be']
#2            Toby             Sam             2000  ['My generation']
#3            Max              Bob             2000  ['Meet the beatles', 'Help']

In [ ]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS music_store (transaction_id int, customer_name text, cashier_name text, year int, albuns_purchased text[]);")
except psycopg2.Error as e: 
    print(e)

In [ ]:
# insert some shit
try: 
    cur.execute("INSERT INTO music_store (transaction_id, customer_name, cashier_name, yeaR, albuns_purchased) VALUES (%s,%s,%s,%s,%s)", (1, "Amanda", "Sam", 2000, ['Rubber Soul', 'Let it be']))
    cur.execute("INSERT INTO music_store (transaction_id, customer_name, cashier_name, yeaR, albuns_purchased) VALUES (%s,%s,%s,%s,%s)", (2, "Toby", "Sam", 2000, ['My generation']))
    cur.execute("INSERT INTO music_store (transaction_id, customer_name, cashier_name, yeaR, albuns_purchased) VALUES (%s,%s,%s,%s,%s)", (3, "Max", "Bob", 2000, ['Meet the beatles', 'Help']))
except psycopg2.Error as e: 
    print(e)

In [ ]:
# check if it works
try: 
    cur.execute("SELECT * FROM music_store")
except psycopg2.Error as e: 
    print(e)

rows = cur.fetchall()
for row in rows:
  print(row)


(1, 'Amanda', 'Sam', 2000, ['Rubber Soul', 'Let it be'])
(2, 'Toby', 'Sam', 2000, ['My generation'])
(3, 'Max', 'Bob', 2000, ['Meet the beatles', 'Help'])


In [ ]:
# Ok, this is not in the 1NF because there is a list of data in the table, moving to 
#the 1NF we must remove the list and therefore make that each data is UNIQUE, in the example make each albun to be on a single row
#Tr Id     Customer Name    Cashier Name       Year    Albuns Purchased
#1            Amanda           Sam             2000    'Rubber Soul'
#1            Amanda           Sam             2000    'Let it be'
#2            Toby             Sam             2000    'My generation'
#3            Max              Bob             2000    'Meet the beatles'
#3            Max              Bob             2000    'Help'

In [ ]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS one_nf_music_store (transaction_id int, customer_name text, cashier_name text, year int, albun_purchased text);")
except psycopg2.Error as e: 
    print(e)

# insert the shit
try: 
    cur.execute("INSERT INTO one_nf_music_store (transaction_id, customer_name, cashier_name, year, albun_purchased) VALUES (%s,%s,%s,%s,%s)", (1, "Amanda", "Sam", 2000, "Rubber Soul"))
    cur.execute("INSERT INTO one_nf_music_store (transaction_id, customer_name, cashier_name, year, albun_purchased) VALUES (%s,%s,%s,%s,%s)", (1, "Amanda", "Sam", 2000, "Let it be"))
    cur.execute("INSERT INTO one_nf_music_store (transaction_id, customer_name, cashier_name, year, albun_purchased) VALUES (%s,%s,%s,%s,%s)", (2, "Toby", "Sam", 2000, "My generation"))
    cur.execute("INSERT INTO one_nf_music_store (transaction_id, customer_name, cashier_name, year, albun_purchased) VALUES (%s,%s,%s,%s,%s)", (3, "Max", "Bob", 2000, "Meet the beatles"))
    cur.execute("INSERT INTO one_nf_music_store (transaction_id, customer_name, cashier_name, year, albun_purchased) VALUES (%s,%s,%s,%s,%s)", (3, "Max", "Bob", 2000, "Help"))
except psycopg2.Error as e: 
    print(e)

# check if it works
try: 
    cur.execute("SELECT * FROM one_nf_music_store")
except psycopg2.Error as e: 
    print(e)

rows = cur.fetchall()
for row in rows:
  print(row)


(1, 'Amanda', 'Sam', 2000, 'Rubber Soul')
(1, 'Amanda', 'Sam', 2000, 'Let it be')
(2, 'Toby', 'Sam', 2000, 'My generation')
(3, 'Max', 'Bob', 2000, 'Meet the beatles')
(3, 'Max', 'Bob', 2000, 'Help')


In [ ]:
# Good, then now let's move this shit to the 2NF so that all columns in the table must rely on the Primary Key, i.e. there must be uniqueness on the primary key, 
# in this example we must create a new primary key, on a new table to organize the albuns
# lets create two tables, transactions and albuns sold

#Tr Id     Customer Name    Cashier Name       Year   
#1            Amanda           Sam             2000    
#2            Toby             Sam             2000   
#3            Max              Bob             2000   


#Album Id  Tr Id      Album Name
#1           1       'Let it be'
#2           1       'Rubber Soul'
#3           2       'My generation'
#4           3       'Meet the beatles'
#5           3       'Help'

In [ ]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS transactions (transaction_id int, customer_name text, cashier_name text, year int);")
    cur.execute("CREATE TABLE IF NOT EXISTS albums_sold (albun_id int, transaction_id int, album_name text);")
except psycopg2.Error as e: 
    print (e)
    
try: 
    cur.execute("INSERT INTO transactions (transaction_id, customer_name, cashier_name, year) VALUES (%s, %s, %s, %s)", (1,"Amanda","Sam", 2000))
    cur.execute("INSERT INTO transactions (transaction_id, customer_name, cashier_name, year) VALUES (%s, %s, %s, %s)", (2, "Toby","Sam",2000))
    cur.execute("INSERT INTO transactions (transaction_id, customer_name, cashier_name, year) VALUES (%s, %s, %s, %s)", (3,"Max","Bob", 2000))
except psycopg2.Error as e: 
    print (e)
    
try: 
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (1,1,'Let it be'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (2,1, 'Rubber Soul'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (3,2,'My generation'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (4,3, 'Meet the beatles'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (5,3,'Help'))
except psycopg2.Error as e: 
    print (e)

print("Table: transactions\n")
cur.execute("SELECT * FROM transactions;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("\nTable: albums_sold\n")
cur.execute("SELECT * FROM albums_sold;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Table: transactions

(1, 'Amanda', 'Sam', 2000)
(2, 'Toby', 'Sam', 2000)
(3, 'Max', 'Bob', 2000)

Table: albums_sold

(1, 1, 'Let it be')
(2, 1, 'Rubber Soul')
(3, 2, 'My generation')
(4, 3, 'Meet the beatles')
(5, 3, 'Help')


In [ ]:
#Let's do a `JOIN` on this table so we can get all the information we had in our first Table. 
#We complete the join on the transactions and album_sold tables

try: 
    cur.execute("SELECT * FROM (transactions JOIN albums_sold ON transactions.transaction_id = albums_sold.transaction_id );")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


(1, 'Amanda', 'Sam', 2000, 1, 1, 'Let it be')
(1, 'Amanda', 'Sam', 2000, 2, 1, 'Rubber Soul')
(2, 'Toby', 'Sam', 2000, 3, 2, 'My generation')
(3, 'Max', 'Bob', 2000, 4, 3, 'Meet the beatles')
(3, 'Max', 'Bob', 2000, 5, 3, 'Help')


In [ ]:
#Moving to 3rd Normal Form (3NF)
## Must be in 2nd Normal Form
## No transitive dependencies
## ----> Remember, transitive dependencies you are trying to maintain is that to get from A-> C, you want to avoid going through B.


#Let's check our table for any transitive dependencies. Transactions can remove Cashier Name to its own table, called Employees, which will leave us with 3 tables. Table Name: transactions2 
#column 0: transaction Id
#column 1: Customer Name
#column 2: Employee Id
#column 3: Year

#Table Name: albums_sold
#column 0: Album Id
#column 1: Transaction Id
#column 3: Album Name

#Table Name: employees
#column 0: Employee Id
#column 1: Employee Name

In [ ]:
cur.execute("DELETE FROM employees")

In [ ]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS transactions2 (transaction_id int, customer_name text, employee_id int, year int);")
    cur.execute("CREATE TABLE IF NOT EXISTS employees (employee_id int, employee_name text);")
    cur.execute("CREATE TABLE IF NOT EXISTS albums_sold (albun_id int, transaction_id int, album_name text);")
except psycopg2.Error as e: 
    print (e)
    
try: 
    cur.execute("INSERT INTO transactions2 (transaction_id, customer_name, employee_id, year) VALUES (%s, %s, %s, %s)", (1,"Amanda", 1, 2000))
    cur.execute("INSERT INTO transactions2 (transaction_id, customer_name, employee_id, year) VALUES (%s, %s, %s, %s)", (2, "Toby",1, 2000))
    cur.execute("INSERT INTO transactions2 (transaction_id, customer_name, employee_id, year) VALUES (%s, %s, %s, %s)", (3,"Max", 1, 2000))
except psycopg2.Error as e: 
    print (e)
    
try: 
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (1,1,'Let it be'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (2,1, 'Rubber Soul'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (3,2,'My generation'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (4,3, 'Meet the beatles'))
    cur.execute("INSERT INTO albums_sold (albun_id , transaction_id, album_name) VALUES (%s, %s, %s)", (5,3,'Help'))
except psycopg2.Error as e: 
    print (e)

try: 
    cur.execute("INSERT INTO employees (employee_id, employee_name) VALUES (%s, %s)", (1,"Sam"))
    cur.execute("INSERT INTO employees (employee_id, employee_name) VALUES (%s, %s)", (2, "Bob"))
except psycopg2.Error as e: 
    print (e)

print("Table: transactions2\n")
cur.execute("SELECT * FROM transactions2;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("\nTable: albums_sold\n")
cur.execute("SELECT * FROM albums_sold;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print("\nTable: employees\n")
cur.execute("SELECT * FROM employees;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Table: transactions2

(1, 'Amanda', 1, 2000)
(2, 'Toby', 1, 2000)
(3, 'Max', 1, 2000)

Table: albums_sold

(1, 1, 'Let it be')
(2, 1, 'Rubber Soul')
(3, 2, 'My generation')
(4, 3, 'Meet the beatles')
(5, 3, 'Help')

Table: employees

(1, 'Sam')
(2, 'Bob')


In [ ]:
### TO-DO: Do a `JOIN` on these tables to get all the information in the original first Table. 

In [ ]:
try: 
    cur.execute("SELECT * FROM (transactions2 JOIN albums_sold ON transactions2.transaction_id = albums_sold.transaction_id JOIN employees ON transactions2.employee_id = employees.employee_id);")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Amanda', 1, 2000, 1, 1, 'Let it be', 1, 'Sam')
(1, 'Amanda', 1, 2000, 2, 1, 'Rubber Soul', 1, 'Sam')
(2, 'Toby', 1, 2000, 3, 2, 'My generation', 1, 'Sam')
(3, 'Max', 1, 2000, 4, 3, 'Meet the beatles', 1, 'Sam')
(3, 'Max', 1, 2000, 5, 3, 'Help', 1, 'Sam')


In [ ]:
# ok, that's it, let1s clean the DB

try: 
    cur.execute("DROP table music_store")
    cur.execute("DROP table albums_sold")
    cur.execute("DROP table employees")
    cur.execute("DROP table transactions")
    cur.execute("DROP table transactions2")
except psycopg2.Error as e: 
    print (e)

table "music_store" does not exist



In [ ]:
cur.close()
conn.close()